## Part 11: export data for a web map
michael babb
2025 02 18

In [ ]:
# standard
import os

In [ ]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
# custom
import run_constants as rc
from utils import write_gdf, write_json, keep_largest_geometry, build_gdf_from_geom, check_MultiLineStrings

# load the city sectors

In [ ]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTORS_OUT_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
gdf.shape

In [ ]:
gdf.head()

#  create non-overlapping polygons

In [ ]:
# keep only the concave bulls
p_gdf = gdf.loc[gdf['hull_type'] == 'concave', :]

In [ ]:
# reproject to wgs 84 utm zone 10
p_gdf = p_gdf.to_crs(epsg = 32610)

In [ ]:
p_gdf.plot()

In [ ]:
p_gdf.head()

In [ ]:
# we first need to reproject
# and then: intersect the data so that it's nice and clean. So clean. 

In [ ]:
# here is what needs to be intersected and clipped
# N and CNTR and E and CNTR
g_names = ['N', 'CNTR', 'E']
geom_dict = {}
for gn in g_names:
    geom_dict[gn] = p_gdf.loc[p_gdf['city_sector'] == gn, 'geometry'].iloc[0]

In [ ]:
# time to do an intersection!
# north / center geometry
nc_geom = geom_dict['N'].intersection(geom_dict['CNTR'])
# east / center geometry
ec_geom = geom_dict['E'].intersection(geom_dict['CNTR'])

# the isolated center geometry
c_geom = geom_dict['CNTR'].difference(geom_dict['N'])
c_geom = c_geom.difference(geom_dict['E'])

# we also need only the north and east geometry. Compute the difference
# north / center geometry
nc_diff_geom = geom_dict['N'].difference(geom_dict['CNTR'])
# east / center geometry
ec_diff_geom = geom_dict['E'].difference(geom_dict['CNTR'])


In [ ]:
# gather and create a geodataframe
geom_list = [c_geom, nc_geom, ec_geom, nc_diff_geom, ec_diff_geom]
data_list = ['CNTR', 'NC', 'EC', 'N', 'E']
mod_gdf = gpd.GeoDataFrame(data = {'city_sector':data_list},
                           geometry = geom_list, crs = 32610)

In [ ]:
# clean up the geometries
mod_gdf = keep_largest_geometry(gdf = mod_gdf, group_col_names=['city_sector'])

In [ ]:
mod_gdf.head()

In [ ]:
# add them to the other geodataframe with the good geometry:
p_gdf.head()

In [ ]:
mod_gdf['hull_type'] = 'concave'

In [ ]:
col_names = p_gdf.columns.tolist()
mod_gdf = mod_gdf[col_names]

In [ ]:
# drop existing geometries in the p_gdf GDF
p_gdf = p_gdf.loc[-p_gdf['city_sector'].isin(mod_gdf['city_sector']), :]

In [ ]:
# stack by concat
p_gdf = pd.concat(objs = [p_gdf, mod_gdf], axis = 0)

In [ ]:
p_gdf = p_gdf.to_crs(epsg=4326)

In [ ]:
write_gdf(gdf = p_gdf, output_file_path=rc.OUTPUT_FILE_PATH, output_file_name=rc.S11_NON_OVERLAPPING_CITY_SECTORS_FILE_NAME)

# create inner ring buffers
The inner ring buffers of the different city sectors aren't used in any analysis.
They are used in the webmap to help orient the reader for the different sectors.
In doing so, it will make the within-sector and cross-sector added streets make
more sense.

In [ ]:
# convert to WGS 84 UTM Zone 10 N, for creating the inner-ring buffers
p_gdf = p_gdf.to_crs(epsg = 32610)

In [ ]:
# let's do some fun inner buffering
output_data_list = []
output_geom_list = []

output_line_data_list = []
output_line_geom_list = []
for ir, row in p_gdf.iterrows():
    city_sector = row['city_sector']
    print(city_sector)
    
    # the focal geometry
    geom = row['geometry']
    # the perimeter
    perim = geom.boundary
    # a dictionary to store the previously created buffer
    # important for creating rings
    previous_buff_dict = {}
    # buffer out 10 units at a time. The units are the same as the units of the
    # geometry's coordinate system.
    for i_dist in range(10, 101, 10):
        # buffer the perimeter. This creates geometry that is both on the inside
        # and outside of the input focal geometry
        my_buff = perim.buffer(distance= i_dist)
        # perform an intersection to get only the stuff on the inside.
        my_buff = my_buff.intersection(geom)
        # remove slivers and splinters
        my_buff = build_gdf_from_geom(geom = my_buff,return_geom=True, crs = p_gdf.crs)        
        
        # add this cleaned geometry to the previous buffer dictionatry
        previous_buff_dict[i_dist] = my_buff

        # now, clip it to the previous buffer
        if i_dist > 10:
            previous_buff = previous_buff_dict[i_dist - 10] 
            # the difference is the part that doesn't overlap - this is the 
            # next ring in the series. 
            my_buff = my_buff.difference(previous_buff)

            my_buff = build_gdf_from_geom(geom = my_buff, return_geom=True, crs = p_gdf.crs)        
         
        # this is for the polygon output 
        temp_list = [city_sector, i_dist]
        output_data_list.append(temp_list)
        output_geom_list.append(my_buff)

        # extract the lines for these inner ring buffers. 
        # one-stop shopping
        line_index = 0
        for line_geom in my_buff.boundary.geoms:
            curr_list = temp_list[:]
            curr_list.append(line_index)
            output_line_data_list.append(curr_list)
            output_line_geom_list.append(line_geom)
            line_index += 1

# create the polygon output gdf
output_gdf = gpd.GeoDataFrame(data = output_data_list, geometry = output_geom_list,
                              crs = 'epsg:32610', columns = ['city_sector', 'distance'])

# project to WGS 84
output_gdf = output_gdf.to_crs(epsg = 4326)

# create the line output gdf
output_line_gdf = gpd.GeoDataFrame(data = output_line_data_list, geometry = output_line_geom_list,
                              crs = 'epsg:32610', columns = ['city_sector', 'distance', 'line_index'])
# project to WGS 84
output_line_gdf = output_line_gdf.to_crs(epsg = 4326)

In [ ]:
# save this to disk
write_gdf(gdf = output_gdf, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name=rc.S11_NON_OVERLAPPING_CITY_SECTORS_POLY_INNER_RING_BUFFER_FILE_NAME)
write_gdf(gdf = output_line_gdf, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name=rc.S11_NON_OVERLAPPING_CITY_SECTORS_LINE_INNER_RING_BUFFER_FILE_NAME)

In [ ]:
# save the output_line_gdf as a geojson for use in a webmap.
# we'll filter different distances in the webmap to get the visual appeal just 
# right. 

In [ ]:
temp_output_gdf = output_line_gdf.copy()
temp_output_gdf.columns = ['cp', 'dist', 'li', 'geometry']
temp_output_gdf['cp'] = temp_output_gdf['cp'].str.replace('CNTR', 'C')
cs_json = temp_output_gdf.to_json(drop_id=True, to_wgs84=True)


In [ ]:
# let's save this to a geojson
write_json(json_data=cs_json, output_file_path='../maps', output_file_name='city_sector_lines.geojson',
           var_name = None)

# export street data for use in a web map

In [ ]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S05_MISSING_IN_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
# dissolve - this also aggregates
col_names = ['ord_stname_type_group','snd_group', 'street_status', 'group_id', 'dist_miles', 'geometry']
diss_gdf = gdf[col_names].dissolve(by = col_names[:-2],
                     aggfunc =  ['sum'], as_index = False)

In [ ]:
# set column names
col_names = ['ord_stname_type_group','snd_group', 'street_status', 'group_id', 'geometry', 'dist_miles']
diss_gdf.columns = col_names

In [ ]:
# add a column to count the number of records - this will be summed later
diss_gdf['n_segments'] = 1
col_names = ['osntg','sndg', 'ss', 'gi', 'geometry', 'dm', 'ns']
diss_gdf.columns = col_names

In [ ]:
# let's try dropping some columns
drop_col_names = ['sndg', 'gi']
diss_gdf = diss_gdf.drop(labels=drop_col_names, axis = 1)

In [ ]:
# perform another dissolve in order to get the righ count of segments
# this is necessary because we want continuous segments to be counted as one
diss_gdf.columns
col_names = ['osntg', 'ss', 'dm', 'ns']
diss_gdf = diss_gdf.dissolve(by = col_names[:2], aggfunc =  ['sum'], as_index = False)
diss_gdf.head()

In [ ]:
# rename 
diss_gdf.columns = ['osntg', 'ss', 'geometry', 'dm', 'ns']
diss_gdf.shape
diss_gdf.head()

In [ ]:
# format the osntg output
def format_osntg(sn):
    if '_' in sn:
        pos = sn.rfind(' ') 
        osn = sn[:pos]
        #ost = sn[pos + 1:].replace('_', '] [')
        #outcome = osn + ' : [' + ost + ']'
        ost = sn[pos + 1:].replace('_', ' | ')
        outcome = osn + ': ' + ost

    else:
        outcome = sn
    return outcome


In [ ]:
diss_gdf['osntg'] = diss_gdf['osntg'].map(format_osntg)
diss_gdf.head()

In [ ]:
# see if we can collapse MultiLineStrings to LineStrings
diss_gdf['geometry'].geom_type.value_counts()


In [ ]:
diss_gdf['geometry'] = diss_gdf['geometry'].map(check_MultiLineStrings)
diss_gdf['geometry'].geom_type.value_counts()

In [ ]:
# finally, rename the CNTR streets to just C
diss_gdf['osntg'] = diss_gdf['osntg'].str.replace('CNTR', 'C')
out_data = diss_gdf.to_json(drop_id=True, to_wgs84=True)
output_file_name = 'all_streets_diss_v2.geojson'
write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name, var_name = 'all_streets_diss')
